In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import json
from datetime import datetime
from functools import partial
import os

todo_items = []

# 레이아웃
layout_label = widgets.Layout(width='80px')
layout_input = widgets.Layout(width='220px')
layout_button = widgets.Layout(width='100%', height='36px')

progress_bar = widgets.FloatProgress(min=0, max=100, value=0, layout=widgets.Layout(width='80%', height='8px'), style={'bar_color': 'black'})
progress_label = widgets.Label(value='(0%)')

todo_list_box = widgets.VBox(layout=widgets.Layout(border='1px solid #ccc', padding='10px'))

def update_overall_progress():
    if not todo_items:
        progress_bar.value = 0
        progress_label.value = '(0%)'
        return
    avg = sum(i['progress'] for i in todo_items) / len(todo_items)
    progress_bar.value = avg
    progress_label.value = f"({int(avg)}%)"

def refresh_list():
    todo_list_box.children = []
    for item in todo_items:
        title = widgets.Text(value=item['title'], layout=layout_input)
        bar = widgets.FloatProgress(value=item['progress'], min=0, max=100, style={'bar_color': 'blue'}, layout=widgets.Layout(width='100%', height='8px'))
        slider = widgets.IntSlider(value=item['progress'], min=0, max=100, step=5, layout=widgets.Layout(width='100%'))
        date = widgets.Label(value=item['due'].strftime('%Y-%m-%d') if item['due'] else '-')
        priority = widgets.Label(value=item['priority'])
        done_btn = widgets.Button(description='완료✅', layout=widgets.Layout(width='100%', height='28px'), button_style='')
        del_btn = widgets.Button(description='삭제❌', layout=widgets.Layout(width='100%', height='28px'), button_style='')

        def on_progress(change, i=item):
            i['progress'] = change['new']
            bar.value = change['new']
            update_overall_progress()

        def on_done(b, i=item):
            i['progress'] = 100
            update_overall_progress()
            refresh_list()

        def on_delete(b, i=item):
            todo_items.remove(i)
            update_overall_progress()
            refresh_list()

        slider.observe(on_progress, names='value')
        done_btn.on_click(partial(on_done, i=item))
        del_btn.on_click(partial(on_delete, i=item))

        row = widgets.HBox([
            widgets.VBox([
                widgets.HBox([widgets.Label('할 일', layout=layout_label), title]),
                widgets.HBox([widgets.Label('진행률', layout=layout_label), bar]),
                widgets.HBox([widgets.Label('', layout=layout_label), slider]),
                widgets.HBox([widgets.Label('마감일', layout=layout_label), date]),
                widgets.HBox([widgets.Label('중요도', layout=layout_label), priority]),
                widgets.VBox([done_btn, del_btn])
            ])
        ])
        todo_list_box.children += (row,)

title_input = widgets.Text(placeholder='할 일을 입력하세요', layout=layout_input)
progress_input = widgets.IntSlider(min=0, max=100, step=5, value=0, layout=layout_input)
due_input = widgets.DatePicker(layout=layout_input)
priority_input = widgets.Dropdown(options=['낮음', '보통', '높음'], value='보통', layout=layout_input)

def add_task(b):
    todo_items.append({
        'title': title_input.value,
        'progress': progress_input.value,
        'due': due_input.value,
        'priority': priority_input.value
    })
    title_input.value = ''
    progress_input.value = 0
    due_input.value = None
    priority_input.value = '보통'
    update_overall_progress()
    refresh_list()

add_btn = widgets.Button(description='추가', layout=layout_button, button_style='success')
add_btn.on_click(add_task)

save_btn = widgets.Button(description='저장', layout=layout_button, button_style='info')
load_btn = widgets.Button(description='불러오기', layout=layout_button, button_style='warning')

def save_data(b):
    with open('todo_data.json', 'w') as f:
        json.dump([{**i, 'due': i['due'].strftime('%Y-%m-%d') if i['due'] else None} for i in todo_items], f)

def load_data(b):
    try:
        with open('todo_data.json', 'r') as f:
            raw = json.load(f)
            todo_items.clear()
            for i in raw:
                todo_items.append({
                    'title': i['title'],
                    'progress': i['progress'],
                    'due': datetime.strptime(i['due'], '%Y-%m-%d').date() if i['due'] else None,
                    'priority': i.get('priority', '보통')
                })
            update_overall_progress()
            refresh_list()
    except:
        print('불러오기 실패')

save_btn.on_click(save_data)
load_btn.on_click(load_data)

display(widgets.VBox([
    widgets.Label('DAVID mk22 - 감성 미니멀 스타일 반영'),
    widgets.HBox([widgets.Label('할 일', layout=layout_label), title_input]),
    widgets.HBox([widgets.Label('진행률', layout=layout_label), progress_input]),
    widgets.HBox([widgets.Label('마감일', layout=layout_label), due_input]),
    widgets.HBox([widgets.Label('중요도', layout=layout_label), priority_input]),
    add_btn,
    widgets.HBox([save_btn, load_btn]),
    widgets.HBox([progress_bar, progress_label]),
    todo_list_box
]))

update_overall_progress()
refresh_list()